In [76]:
from py2neo import Graph, Path
import numpy as np
import pandas as pd

In [77]:
graph = Graph('http://localhost:7474/')

In [78]:
with open('../dat/output_edges.csv', 'r') as f:
    raw_edges = f.read().splitlines()
    
edge_arr = np.zeros((len(raw_edges), 2))
for i in range(len(raw_edges)):
    temp = raw_edges[i].split(',')
    edge_arr[i, 0] = int(temp[0])
    edge_arr[i, 1] = int(temp[1])

In [79]:
raw_dat = pd.read_csv('../dat/output.csv')
node_dat = raw_dat[['Cluster ID', 'Id', 'Site name', 'Address', 'Zip', 'Phone']].sort_values(by='Id')
node_dat.head()

,Cluster ID,Id,Site name,Address,Zip,Phone
0,0,0,salvation army - temple / salvation army,1 n ogden ave,NaN,2262649.0
1,0,1,salvation army - temple / salvation army,1 n ogden ave,NaN,2262649.0
10,1,2,national louis university - dr. effie o. ellis...,10 s kedzie ave,NaN,5339011.0
11,1,3,national louis university - dr. effie o. ellis...,10 s kedzie ave,NaN,5339011.0
20,2,4,board trustees-city colleges of chicago - oliv...,10001 s woodlawn ave,NaN,2916100.0


In [80]:
np.amin(node_dat['Id'] == range(node_dat.shape[0]))

False

In [81]:
%%time

tx = graph.begin(autocommit=False)

statement = "CREATE (n:ids {school:{SCHOOLID}, cluster:{CLUSTERID}})"

for i in range(len((node_dat))):
    tx.run(statement, {'SCHOOLID': float(node_dat.iloc[i]['Id']),
                         'CLUSTERID': float(node_dat.iloc[i]['Cluster ID']),
                        })

tx.commit()

tx = graph.begin(autocommit=False)

statement = "CREATE (n:addresses {address:{ADDR}})"

for i in node_dat['Address'].unique():
    tx.run(statement, {'ADDR': i})

tx.commit()

tx = graph.begin(autocommit=False)

statement = "CREATE (n:names {name:{SCHOOLNAME}})"

for i in node_dat['Site name'].unique():
    tx.run(statement, {'SCHOOLNAME': i})

tx.commit()

tx = graph.begin(autocommit=False)

statement = "CREATE (n:zipcodes {zip:{SCHOOLZIP}})"

for i in node_dat['Zip'].unique():
    tx.run(statement, {'SCHOOLZIP': i})

tx.commit()

tx = graph.begin(autocommit=False)

statement = "CREATE (n:phones {phone:{SCHOOLPHONE}})"

for i in node_dat['Phone'].unique():
    tx.run(statement, {'SCHOOLPHONE': i})

tx.commit()

CPU times: user 2.37 s, sys: 12 ms, total: 2.38 s
Wall time: 2.67 s


In [82]:
graph.data("MATCH (n) RETURN n LIMIT 10;")

[{'n': (f643ed2:ids {cluster:0.0,school:0.0})},
 {'n': (b758262:ids {cluster:0.0,school:1.0})},
 {'n': (f0ec0a5:ids {cluster:1.0,school:2.0})},
 {'n': (b9a8339:ids {cluster:1.0,school:3.0})},
 {'n': (a2fd0db:ids {cluster:2.0,school:4.0})},
 {'n': (a8b65e1:ids {cluster:2.0,school:5.0})},
 {'n': (b731599:ids {cluster:3.0,school:6.0})},
 {'n': (ee5811a:ids {cluster:3.0,school:7.0})},
 {'n': (d2e49c9:ids {cluster:4.0,school:8.0})},
 {'n': (c903bf0:ids {cluster:5.0,school:9.0})}]

In [83]:
graph.data('MATCH (n:zipcodes {zip:60608}) RETURN n LIMIT 10')

[{'n': (ab3d116:zipcodes {zip:60608.0})}]

In [84]:
#graph.run('MATCH (n) DELETE n')
#graph.data('MATCH (n) RETURN n;')

In [85]:
%%time

tx = graph.begin(autocommit=False)

statement = "MATCH (a:ids),(b:addresses) \
           WHERE a.school = {ID} AND \
           b.addresses = {ADDR} \
           CREATE (a)-[r:HAS_ADDRESS]->(b) \
           RETURN r"

for i in range(len(node_dat)):
    tx.run(statement, {'ID': float(node_dat.iloc[i]['Id']),
                      'ADDR': str(node_dat.iloc[i]['Address'])})

tx.commit()

tx = graph.begin(autocommit=False)

statement = "MATCH (a:ids),(b:names) \
           WHERE a.school = {ID} AND \
           b.name = {NAME} \
           CREATE (a)-[r:HAS_NAME]->(b) \
           RETURN r"

for i in range(len(node_dat)):
    tx.run(statement, {'ID': float(node_dat.iloc[i]['Id']),
                      'NAME': str(node_dat.iloc[i]['Site name'])})

tx.commit()

tx = graph.begin(autocommit=False)

statement = "MATCH (a:ids),(b:zipcodes) \
           WHERE a.school = {ID} AND \
           b.zip = {ZIP} \
           CREATE (a)-[r:HAS_ZIP]->(b) \
           RETURN r"

for i in range(len(node_dat)):
    tx.run(statement, {'ID': float(node_dat.iloc[i]['Id']),
                      'ZIP': str(node_dat.iloc[i]['Zip'])})

tx.commit()

tx = graph.begin(autocommit=False)

statement = "MATCH (a:ids),(b:phones) \
           WHERE a.school = {ID} AND \
           b.phone = {PHONE} \
           CREATE (a)-[r:HAS_NAME]->(b) \
           RETURN r"

for i in range(len(node_dat)):
    tx.run(statement, {'ID': float(node_dat.iloc[i]['Id']),
                      'PHONE': str(node_dat.iloc[i]['Phone'])})

tx.commit()

CPU times: user 6.96 s, sys: 80 ms, total: 7.04 s
Wall time: 14.8 s


In [86]:
#graph.run('MATCH (n) DETACH DELETE n')
#graph.data('MATCH (n) RETURN n')

In [90]:
graph.data('MATCH (a:ids)-[:HAS_NAME]->(b:names) \
            WHERE a.school = 2 \
            RETURN b')

[{'b': (e85d1a2:names {name:"national louis university - dr. effie o. ellis training center"})}]